## Get Cagr

In [ ]:
# This script iterates over json file with ETF's monthly values, gets the funds CAGR and  

import pandas as pd
import json
from datetime import datetime


with open("sample-monthly-values.json", "r") as file:
    data = json.load(file) 


for etf in data:
    monthly_values = data[etf]["Monthly Adjusted Time Series"]
    
    monthly_values_keys = list(monthly_values.keys())

    current_date = datetime.strptime(monthly_values_keys[0],  "%Y-%m-%d").date()
    start_date = datetime.strptime(monthly_values_keys[-1],  "%Y-%m-%d").date()
    
    duration = (current_date - start_date) # duration is a datetime object 

    fund_years = duration.days / 365.25
    
    current_etf_price = float(monthly_values[monthly_values_keys[0]]["4. close"])

    start_etf_price = float(monthly_values[monthly_values_keys[-1]]["4. close"])

    cagr = ((current_etf_price / start_etf_price)**(1/fund_years)) - 1

    data[etf]["Meta Data"]["5. CAGR"] = cagr

    print(f"{etf} CAGR: ", cagr)

print(data['ARKK']["Meta Data"])

# Store data in JSON file
with open('sample-monthly-values.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)



## Annualized Returns Table
- this script csv table with all the monthly values & uses pandas to return a csv table of each fund's yearly returns. 

In [ ]:
import pandas as pd


# Import CSV with Monthly Values 
monthly_values_df = pd.read_csv("sample-monthly-values.csv")

# Initialize Empty Dataframe
yearly_returns_df = pd.DataFrame()

# Convert the 'timestamp' column to datetime
monthly_values_df['timestamp'] = pd.to_datetime(monthly_values_df['timestamp'])

# Set the 'timestamp' as the DataFrame index
monthly_values_df.set_index('timestamp', inplace=True)

# Group data by 'ticker' if you have multiple ETFs in the CSV
etf_groups = monthly_values_df.groupby('ticker')  # creates two dataframes: one for each ticker


# Iterate over each ETF group
for ticker, group in etf_groups:
    # Resample the data by year, selecting the first and last 'close' price for each year
    # returns a dataframe with: timestamp / first (price) / last (price) 
    annual_data = group.resample('Y').agg({
        'close': ['first', 'last']  # take the first and last closing prices for each year 
    })
    
    # Calculate annualized returns
    annual_data['annualized_return'] = (annual_data[('close', 'last')] / annual_data[('close', 'first')]) - 1
    annual_data['ticker'] = ticker

    # Normalize column names
    annual_data.columns = ['first', 'last', 'annualized_return', 'ticker']

    yearly_returns_df = pd.concat([yearly_returns_df, annual_data], ignore_index=False)


# Save yearly returns in csv 
yearly_returns_df.to_csv('sample-yearly-returns.csv', index=True)



## Get Standard Deviation of Annual Returns

In [ ]:
import pandas as pd


# Import yearly returns csv table 
yearly_returns_table = pd.read_csv("sample-yearly-returns.csv")

# 1. Calculate each fund's Standard Deviation of annual returns 
stdev = yearly_returns_table.groupby('ticker')["annualized_return"].std()

print(stdev)

# 1.2 Export table with Standard Deviations to csv 
stdev.to_csv("sample-standard-deviations.csv", index=True)


# 2. Import monthly values file 
with open("sample-monthly-values.json", "r") as file:
    monthly_values_json = json.load(file) 

# 2.2 Store Standard Devs in Monthly Values File 
for ticker in stdev.index:
    monthly_values_json[ticker]["Meta Data"]["6. Stdev of Returns"] = stdev[ticker]

print(monthly_values_json['ARKK']["Meta Data"])

# 2.3 Update monthly values file with STDev data
with open('sample-monthly-values.json', 'w') as json_file:
    json.dump(monthly_values_json, json_file, indent=4)
 


## Get Sharpe Ratio

In [ ]:
risk_free_rate = 0.02

file_dict = {
    "ARKK": {
        "Meta Data": {
            "1. Information": "Monthly Adjusted Prices and Volumes",
            "2. Symbol": "ARKK",
            "3. Last Refreshed": "2024-09-23",
            "4. Time Zone": "US/Eastern",
            "5. CAGR": 0.08753236094442607,
            "6. Stdev of Returns": 0.5249136603898312
        }},
    "AZTD": {
        "Meta Data": {
            "1. Information": "Monthly Adjusted Prices and Volumes",
            "2. Symbol": "AZTD",
            "3. Last Refreshed": "2024-09-23",
            "4. Time Zone": "US/Eastern",
            "5. CAGR": 0.16460331337349632,
            "6. Stdev of Returns": 0.05367809475589438
        }}}

for etf in file_dict:
    file_dict[etf]["Meta Data"]["7. Sharpe Ratio"] = (file_dict[etf]["Meta Data"]["5. CAGR"] - risk_free_rate) / file_dict[etf]["Meta Data"]["6. Stdev of Returns"]

print(file_dict["ARKK"]["Meta Data"])


In [ ]:
import json

# Import monthly values file 
with open("sample-monthly-values.json", "r") as file:
    monthly_values_json = json.load(file) 

risk_free_rate = 0.02

for etf in monthly_values_json:
    monthly_values_json[etf]["Meta Data"]["7. Sharpe Ratio"] = (monthly_values_json[etf]["Meta Data"]["5. CAGR"] - risk_free_rate) / monthly_values_json[etf]["Meta Data"]["6. Stdev of Returns"]


monthly_values_json["AZTD"]["Meta Data"]

# Store data in JSON file
with open('sample-monthly-values-final.json', 'w') as json_file:
    json.dump(monthly_values_json, json_file, indent=4)